# Building a Conversational Chatbot for Slack using Rasa and Python -Part 1

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

# Installations
* Rasa NLU
* Rasa Core
* SpaCy Language Mode

In [2]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core rasa_nlu[spacy];

Requirement already up-to-date: rasa_core in /Users/macbook/opt/anaconda3/lib/python3.7/site-packages (0.14.5)
Requirement already up-to-date: rasa_nlu[spacy] in /Users/macbook/opt/anaconda3/lib/python3.7/site-packages (0.15.1)


In [3]:
!{python} -m spacy download en_core_web_md


    Linking successful
    /Users/macbook/opt/anaconda3/lib/python3.7/site-packages/en_core_web_md
    -->
    /Users/macbook/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



## Downloading the English Language Model

In [4]:
!{python} -m spacy link en_core_web_md en --force;


    Linking successful
    /Users/macbook/opt/anaconda3/lib/python3.7/site-packages/en_core_web_md
    -->
    /Users/macbook/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



# Importing the Installations

In [5]:
import rasa_nlu
import rasa_core
import spacy

# 1. Teaching the bot to understand user inputs using Rasa NLU

## Preparing the NLU Training Data

In [6]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- Quite bad - can I get a cute picture of a [bird](group:birds), please?
- Really bad and only [doggo](group:shibes) pics and change that.
- Not good. The only thing that could make me fell better is a picture of a cute [kitten](group:cats).
- so sad. Only the picture of a [puppy](group:shibes) could make it better.
- I am very sad. I need a [cat](group:cats) picture.
- Extremely sad. Only the cute [doggo](group:shibes) pics can make me feel better.
- Bad. Please show me a [bird](group:birds) pic!
- Pretty bad to be honest. Can you show me a [puppy](group:shibes) picture to make me fell better?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


# Defining the NLU Model Configuration

In [7]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [8]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'inform', 'goodbye', 'mood_unhappy', 'greet', 'mood_deny', 'mood_affirm', 'mood_great'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.model:Starting to train component SpacyNLP
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train c

Fitting 2 folds for each of 6 candidates, totalling 12 fits


INFO:rasa_nlu.model:Successfully saved model into '/Users/macbook/dataV2-labs/Projects/models/nlu/default/current'


## Evaluating the NLU model on a random text

In [9]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("I am very sad. Could you send me a cat picture? "))

{
  "intent": {
    "name": "mood_unhappy",
    "confidence": 0.6410226269696062
  },
  "entities": [
    {
      "start": 35,
      "end": 38,
      "value": "cats",
      "entity": "group",
      "confidence": 0.9543162015604569,
      "extractor": "CRFEntityExtractor",
      "processors": [
        "EntitySynonymMapper"
      ]
    }
  ],
  "intent_ranking": [
    {
      "name": "mood_unhappy",
      "confidence": 0.6410226269696062
    },
    {
      "name": "goodbye",
      "confidence": 0.10918827027441855
    },
    {
      "name": "mood_great",
      "confidence": 0.09930106946218047
    },
    {
      "name": "greet",
      "confidence": 0.05768743638451059
    },
    {
      "name": "inform",
      "confidence": 0.034004823927633626
    },
    {
      "name": "mood_affirm",
      "confidence": 0.03353057501666721
    },
    {
      "name": "mood_deny",
      "confidence": 0.025265197964983447
    }
  ],
  "text": "I am very sad. Could you send me a cat picture? "
}


## Evaluating the NLU model on a test data
### (Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [10]:
from rasa_nlu.test import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- Found intents: 'inform', 'goodbye', 'mood_unhappy', 'greet', 'mood_deny', 'mood_affirm', 'mood_great'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.test:Running model for predictions:
100%|██████████| 85/85 [00:00<00:00, 132.29it/s]
INFO:rasa_nlu.test:Intent evaluation results:
INFO:rasa_nlu.test:Intent Evaluation: Only considering those 85 examples that have a defined intent out of 85 examples
INFO:rasa_nlu.test:F1-Score:  0.9881730469965763
INFO:rasa_nlu.test:Precision: 0.9890756302521008
INFO:rasa_nlu.test:Accuracy:  0.9882352941176471
INFO:rasa_nlu.test:Classification report: 
              precision    recall  f1-score   support

     goodbye       1.00      0.91      0.95        

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.6100184030407697},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5059332984288191},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.6438979894288671},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5059332984288191},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5394322521259962},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5385969731121552},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5258913869740606},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5025940468321921},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'gre

# 2. Teaching the bot to respond using Rasa Core
## 1. Writing Stories

In [11]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_ask_picture
* inform{"animal":"dog"}  
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_ask_picture
* inform{"group":"cat"}
  - action_retrieve_image
  - utter_did_that_help
* mood_deny
  - utter_goodbye
  
## sad path 3
* greet
  - utter_greet
* mood_unhappy{"group":"puppy"}
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


## 2. Defining a Domain

In [12]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

slots:
  group:
    type: text
    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture
- __main__.ApiAction

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


# Custom Actions

In [13]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        
        r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))

# Visualising the Training Data

In [17]:
!{python} -m pip install pygraphviz;

/bin/sh: apt-get: command not found
/bin/sh: breq: command not found


In [20]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 277.76it/s, # trackers=1]


AttributeError: 'MultiDiGraph' object has no attribute 'node'

# Training a Dialogue Model

In [24]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

Processed Story Blocks:   0%|          | 0/7 [00:00<?, ?it/s, # trackers=11]WARNING:rasa_core.domain:Failed to use action 'action_retrieve_image' in history. Please make sure all actions are listed in the domains action list. If you recently removed an action, don't worry about this warning. It should stop appearing after a while. 


Processed Story Blocks:   0%|          | 0/7 [00:00<?, ?it/s, # trackers=12]WARNING:rasa_core.domain:Failed to use action 'action_retrieve_image' in history. Please make sure all actions are listed in the domains action list. If you recently removed an action, don't worry about this warning. It should stop appearing after a while. 


Processed Story Blocks:   0%|          | 0/7 [00:00<?, ?it/s, # trackers=12]WARNING:rasa_core.domain:Failed to use action 'action_retrieve_image' in history. Please make sure all actions are listed in the domains action list. If you recently removed an action, don't worry about this warning. It should stop appearing after a while. 


Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 57.69it/s, # trackers=12]


Exception: Passing policy configuration parameters to `agent.train(...)` is not supported anymore. Specify parameters directly in the policy configuration instead. More info https://legacy-docs.rasa.com/docs/core  /migrations.html

In [25]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

Exception: Failed to load domain specification from '/Users/macbook/dataV2-labs/Projects/models/dialogue/domain.yml'. File not found!

In [26]:

print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
yes


Exception: BinarySingleStateFeaturizer was not prepared before encoding.